In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os
import copy

class BasisFilter:
    
    def __init__(self):
        self.path = os.getcwd() 
        self.path =os.path.abspath(os.path.join(self.path, "..", ".."))
        self.newPath = os.path.join(f'{self.path}\\filtered_data')
        self.region_list = ['bundang', 'gangnam', 'hongdae', 'jamsil', 'bucheon', 'busan', 'incheon']
        self.index_level_2= ['Date', 'ProgressNote', 'MedicineName','Memo','체중','골격근량','체지방량','BMI','혈압(고)','혈압(저)','맥박수','체지방률','Height','근육량',
            'InterCellWater','ExtraCellWater','TotalBodyWater','ProteinMass','MineralMass','FatFreeMass','Osseus','ECW_TBW','ECF_TBF','VFA','WHR',
            'WeightControl','FatControl','MuscleControl','BMR','FitnessScore']

    def make_dir(self):
        if not os.path.exists(self.newPath):
            os.makedirs(os.path.join(f'{self.path}\\filtered_data'))
        else:
            print('이미 존재합니다.')

    def del_dir(self):
        # 'PreprocessData' 폴더가 존재하는지 확인하고 삭제
        if os.path.exists(self.newPath) and os.path.isdir(self.newPath):
            try:
                shutil.rmtree(self.newPath)
                print(f"'{self.newPath}' 폴더가 성공적으로 삭제되었습니다.")
            except Exception as e:
                print(f"폴더를 삭제하는 중 오류가 발생했습니다: {e}")
        else:
            print(f"'{self.newPath}' 폴더가 존재하지 않습니다.")

    def load_basis(self):
        self.basis = pd.read_csv(f'{self.path}\\MergeRegionData\\basis_for_filter.csv', encoding='utf-8')
        # date_columns = ['PatientFirstDate'] + ['Date'] + ['Date.'+str(i) for i in range(1,28)]
        # for col in date_columns:
        #     self.basis[col] = pd.to_datetime(self.basis[col],format='%Y%m%d')
        return self.basis

    def first_visit_filter(self, df):
        
        pattern = r'\[1\]\s*기본\s*상담*'
        # 각 ProgressNote 열에서 두 가지 패턴을 모두 만족하는 행 필터링
        filtered_df = df[
            df['ProgressNote_1'].str.contains(pattern, regex=True) |
            df['ProgressNote_2'].str.contains(pattern, regex=True) |
            df['ProgressNote_3'].str.contains(pattern, regex=True) |
            df['ProgressNote_4'].str.contains(pattern, regex=True)
        ]
        # 필터링된 행의 PatientID 추출
        result_patient_ids = filtered_df['PatientID'].tolist()
        filtered_df = filtered_df['2022-11-06'<filtered_df['PatientFirstDate']]
        return filtered_df
        
    def save_csv(self, df, name):
        df.to_csv(f'{self.newPath}\\{name}.csv')
        
    def save_excel(self, df, name):
        df.to_excel(f'{self.newPath}\\{name}.xlsx')

    def load_csv(self, name):
        df = pd.read_csv(f'{self.newPath}\\{name}.csv', encoding ='utf-8')
        return df

    def make_inbody2up_dict(self, df):
        idx_list = df.index.tolist()
        
        Weight_list = ['Weight_'+str(i) for i in range(1,28)]
        inbody2up_dict = {}
        for idx in idx_list:
            inbody_weights = pd.DataFrame(df.loc[idx,Weight_list].dropna()).transpose()
            Date_list = ['Date_' + weight_col.split('_')[-1] for weight_col in inbody_weights.columns.tolist()]
            inbody_dates = df.loc[idx,Date_list]
            if len(inbody_dates) > 1:
                inbody2up_dict[idx] = list(inbody_dates)
        self.inbody2up_idx = list(inbody2up_dict.keys())
        self.inbody2up_dict = inbody2up_dict
        return self.inbody2up_dict

    def make_return_dict(self, df, dict, day_range): #day range는 리스트형태로 [a, b]의 형태로 입력
        return_dict = {}
        for idx in dict.keys():
            for date in dict[idx][1:]:
                day = (datetime.strptime(date, '%Y-%m-%d') - datetime.strptime(dict[idx][0], '%Y-%m-%d')).days
                if day_range[0] < day < day_range[1]:
                    return_dict[idx] = date
        self.return_dict = return_dict
        self.return_idx = list(return_dict.keys())
        return self.return_dict

    def make_medicine_dict(self, df, inbody2up_dict, return_dict):
        #record_dict은 gambi에 대한 정보만을 저장
        #allmed는 같이 사용된 약의 정보까지 모두 저장
        self.record_dict ={}
        self.allmed_dict ={}
        for idx in return_dict.keys():
            return_day = return_dict[idx]
            return_day = datetime.strptime(return_day, '%Y-%m-%d')
            first_day = inbody2up_dict[idx][0]
            first_day = datetime.strptime(first_day, '%Y-%m-%d')
            target = 0
            record_list =[]
            med_list = []
            
            for i in range(1,32):
                med = f'MedicineName_{i}'
                date = f'Date_{i}'
                memo = f'Memo_{i}'
                if pd.isna(df.loc[idx,date]):
                    continue        
                day = datetime.strptime(df.loc[idx,date], '%Y-%m-%d')
                if return_day - timedelta(days=1) <= day:
                    break
                if first_day - timedelta(days=1) > day:
                    break
                if pd.isna(df.loc[idx,med]):
                    continue
                else:
                    for j, medicine in enumerate(eval(df.loc[idx,med])):
                        if 'Gambi' in medicine:
                            if 'Tab' in medicine:
                                memo_list =eval(df.loc[idx,memo])[j]
                                if memo_list =='2-1':
                                    record_list.append([day,medicine,'2-1'])
                                    target = target + 1
                                    med_list.append(eval(df.loc[idx,med]))
                                if memo_list =='2-2':
                                    record_list.append([day,medicine,'2-2'])
                                    target = target + 1
                                    med_list.append(eval(df.loc[idx,med]))
            if target > 1:
                #print(idx, target, return_day)
                self.record_dict[idx] = record_list
                self.allmed_dict[idx] = med_list

    def make_FR_df(self, df, medicine_idx, return_dict, inbody2up_dict):
        
        concat_df = pd.DataFrame()
        df_info = df[['Region','PatientChartNo','PatientAddr11','PatientFirstDate','Age','PatientSex','Description']]
        dates = ['Date_'+str(i) for i in range(1,28)]
        for idx in medicine_idx:
            #print(inbody2up_dict[idx][0],return_dict[idx])
            F = int(df[dates].loc[idx, df.loc[idx] == inbody2up_dict[idx][0]].index.tolist()[0].split('_')[-1])
            R = int(df[dates].loc[idx, df.loc[idx] == return_dict[idx]].index.tolist()[0].split('_')[-1])
            df_info_i = pd.DataFrame(df_info.loc[idx]).transpose()
            #print(df_info_i)
            Fs_idx = df.columns.get_loc('Date_'+str(F))
            Fl_idx = df.columns.get_loc('Date_'+str(F+1))
            Rs_idx = df.columns.get_loc('Date_'+str(R))
            Rl_idx = df.columns.get_loc('Date_'+str(R+1))
            F_df = pd.DataFrame(df.iloc[:, Fs_idx:Fl_idx].loc[idx]).transpose()
            F_col =F_df.columns.tolist()
            F_col =['_'.join(col.split('_')[:-1]+['F']) for col in F_col]
            F_df = F_df.set_axis(F_col, axis = 'columns')
            R_df = pd.DataFrame(df.iloc[:, Rs_idx:Rl_idx].loc[idx]).transpose()
            R_col =R_df.columns.tolist()
            R_col =['_'.join(col.split('_')[:-1]+['R']) for col in R_col]
            R_df = R_df.set_axis(R_col, axis = 'columns')
            FR_df = pd.concat([F_df,R_df], axis = 1)
            FR_df = pd.concat([df_info_i,FR_df], axis = 1)
        
            concat_df = pd.concat([concat_df,FR_df], axis = 0)
        self.FR_df = concat_df
        return self.FR_df
            
        


In [2]:
BF = BasisFilter()

In [3]:
basis = BF.load_basis()

C:\Users\howsr\AppData\Local\Temp\ipykernel_44532\1751246202.py:36: DtypeWarning: Columns (2,68,69,70,71,98,99,100,101,128,129,130,131,158,159,160,161,188,189,190,191,218,219,220,221,248,249,250,251,278,279,280,281,308,309,310,311,338,339,340,341,368,369,370,371,398,399,400,401,428,429,430,431,458,459,460,461,488,489,490,491,518,519,520,521,548,549,550,551,578,579,580,581,608,609,610,611,638,639,640,641,668,669,670,671,698,699,700,701,728,729,730,731,758,759,760,761,788,789,790,791) have mixed types. Specify dtype option on import or set low_memory=False.
  self.basis = pd.read_csv(f'{self.path}\\MergeRegionData\\basis_for_filter.csv', encoding='utf-8')


In [4]:
BF.basis

,Region,PatientID,PatientChartNo,PatientAddr11,PatientFirstDate,Age,PatientSex,Description,Date_1,ProgressNote_1,...,Osseus_27,ECW_TBW_27,ECF_TBF_27,VFA_27,WHR_27,WeightControl_27,FatControl_27,MuscleControl_27,BMR_27,FitnessScore_27
0,bundang,8,14634,"경기도 성남시 수정구 산성대로437번길 7 (단대동, 푸르지오)",2014-06-30,40.0,2,NaN,2022-11-25,"#비만\n\n신청 약 : 감비정 - D2X 405, 팻아웃 3, 부스터정 1\n차수...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bundang,15,25008,전라북도 군산시 하나운2길 15(나운동),2016-01-22,21.0,2,추천인 길갑인(7744)님.,2022-11-24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bundang,26,55309,서울특별시 용산구 청파로 205-6(문배동),2018-03-19,24.0,2,NaN,2022-11-23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bundang,34,41894,"세종특별자치시 달빛로 165(아름동, 범지기마을 8단지)",2017-04-10,38.0,2,NaN,2022-12-05,"#비만\n\n신청 약 : 감비정M 405, 팻아웃플러스 1\n차수 : 2-2\n\n...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bundang,43,43102,"경기도 용인시 기흥구 중부대로746번길 21 (상하동, 지석마을진흥더루벤스2단지아파트)",2017-05-22,59.0,2,소개자 다음내원시 확인(전광희님),2023-06-28,# 비만\n\n현재체중 : 68\n마지막 약 복용일자 : 1년이상 \n포만감&식사조...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130075,incheon,21816,3954.0,"인천광역시 연수구 송도문화로28번길 27 (송도동, 송도글로벌파크베르디움)",2024-05-14,43.0,2,분당25851->인천,2024-05-14,#비만(전화상담접수)\n\n★마지막처방일 5년경과 의료진 확인사항 \n현재체중 : ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130076,incheon,21821,3959.0,"경기도 동두천시 이담로 161 (지행동, 동두천 에코 휴먼빌2차 아파트)",2024-05-14,49.0,2,비대면,2024-05-14,#비만(비대면초진)\n\n여자\n\nㆍ 키/체중/BMI:156 / 72 / 경도비만...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130077,incheon,21836,3961.0,"인천광역시 남동구 석산로222번길 53 (구월동, 행복꿈터)",2024-05-14,33.0,2,NaN,2024-05-14,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:162 / 73 / 경도비만(...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130078,incheon,21840,3962.0,"인천광역시 남동구 석산로222번길 53 (구월동, 행복꿈터)",2024-05-14,39.0,1,NaN,2024-05-14,#비만(내원초진)\n\n남자\n\nㆍ 키/체중/BMI:173 / 99.9 / 중도 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
first_df = BF.first_visit_filter(basis)
first_df.shape

(44829, 818)

In [6]:
inbody2up_dict = BF.make_inbody2up_dict(first_df)

In [7]:
inbody2up_df = first_df[first_df.index.isin(list(inbody2up_dict.keys()))]

In [8]:
len(inbody2up_dict)

9739

In [9]:
return_dict = BF.make_return_dict(inbody2up_df, inbody2up_dict, [70,98])

In [41]:
for idx in return_dict.keys():
    print(idx)
    print(inbody2up_dict[idx])
    print(return_dict[idx])
    print()

988
['2023-04-21', '2023-07-03', '2023-10-11', '2024-01-15']
2023-07-03

2720
['2023-03-25', '2023-05-03', '2023-06-09']
2023-06-09

8361
['2023-09-02', '2023-12-02']
2023-12-02

11729
['2023-06-14', '2023-07-19', '2023-09-11', '2023-10-16', '2024-01-10']
2023-09-11

14678
['2023-08-30', '2023-10-11', '2023-11-24', '2024-01-20']
2023-11-24

15341
['2023-07-17', '2023-10-20', '2023-11-17', '2024-01-02', '2024-01-23']
2023-10-20

15580
['2023-09-13', '2023-11-27']
2023-11-27

15833
['2024-01-08', '2024-02-02', '2024-04-12']
2024-04-12

19320
['2023-09-06', '2023-11-20']
2023-11-20

19510
['2023-06-27', '2023-09-08', '2023-12-05', '2024-05-04']
2023-09-08

19951
['2022-12-27', '2023-03-14', '2023-07-10', '2024-03-20']
2023-03-14

20146
['2023-06-26', '2023-08-12', '2023-09-13']
2023-09-13

20916
['2022-11-08', '2023-01-30']
2023-01-30

21030
['2022-11-15', '2023-02-04']
2023-02-04

21048
['2022-11-22', '2023-02-21']
2023-02-21

21176
['2022-11-23', '2022-12-23', '2023-02-28', '2023-04-18'

In [11]:
BF.make_medicine_dict(first_df, inbody2up_dict, return_dict)

In [12]:
record_dict=BF.record_dict
med_idx = list(record_dict.keys())

In [13]:
filtered_df = first_df.iloc[first_df.index.isin(list(record_dict.keys()))]

In [14]:
BF.make_FR_df(first_df,med_idx,return_dict,inbody2up_dict)

,Region,PatientChartNo,PatientAddr11,PatientFirstDate,Age,PatientSex,Description,Date_F,ProgressNote_F,MedicineName_F,...,Osseus_R,ECW_TBW_R,ECF_TBF_R,VFA_R,WHR_R,WeightControl_R,FatControl_R,MuscleControl_R,BMR_R,FitnessScore_R
2720,bundang,118562,"경기도 성남시 분당구 판교로 393(삼평동, 봇들마을2단지이지더원아파트)",2023-03-25,60.0,2,NaN,2023-03-25,#비만\n\n여자\n\nㆍ 키/체중/BMI:158 / 75 / 중도 비만(30.04...,"['GambiTab-D4_1통*3#405', '팻아웃 정1통*1 ']",...,2.45,0.389,0.342,141.4,0.93,-14.4,-14.4,0.0,1314.0,70.0
8361,bundang,122116,"경기도 수원시 영통구 동탄원천로881번길 35 (매탄동, 주공그린빌)",2023-09-02,31.0,2,내원경로 이주희(강45909/ 핸7385),2023-09-02,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:157 / 60 / 과체중(2...,"['GambiTab-D4_1통*3#405', 'GambiTab-D5_1통*3#405...",...,2.28,0.378,0.332,92.1,0.87,-7.7,-8.2,0.5,1202.0,71.0
14678,bundang,122073,경기도 성남시 중원구 여수울로29번길 21 (여수동),2023-08-30,43.0,2,NaN,2023-08-30,#비만\n\n여자\n\nㆍ 키/체중/BMI:166 / 60 / 정상(21.77384...,"['GambiTab-D5_1통*3#405', '팻아웃 정1통*1 ', '비움정 ...",...,2.31,0.386,0.339,65.4,0.85,2.8,-2.0,4.8,1210.0,73.0
15580,bundang,122237,"경기도 성남시 중원구 여수울로 51 (여수동, 산들마을)",2023-09-13,44.0,2,NaN,2023-09-13,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:160 / 82 / 중도 비만...,"['GambiTab-D6_1통*3#405', 'GambiTab-D2X_1통*3#40...",...,2.74,0.377,0.33,147.3,0.97,-15.4,-15.4,0.0,1465.0,74.0
15833,bundang,124008,"경기도 의왕시 바라산로 75 (학의동, 의왕백운 골드클래스 아파트)",2024-01-08,36.0,1,NaN,2024-01-08,#비만(내원초진)\n\n남자\n\nㆍ 키/체중/BMI:184 / 115 / 중도 비...,"['GambiTab-D2X_1통*3#405', 'GambiTab-D3X_1통*3#4...",...,4.16,0.38,0.333,159.2,1.02,-22.2,-22.2,0.0,1874.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128890,incheon,2739.0,"인천광역시 미추홀구 인주대로496번길 42 (주안동, 플러스캐슬)",2024-01-11,49.0,2,NaN,2024-01-11,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:159 / 56 / 정상(22...,"['GambiTab-D5_1통*3#405', 'GambiTab-D6_1통*3#405...",...,2.21,0.394,0.346,91.0,0.83,-4.6,-7.0,2.4,1156.0,71.0
128900,incheon,2784.0,"인천광역시 연수구 한나루로105번길 68 (옥련동, 현대4차아파트)",2024-01-11,44.0,2,NaN,2024-01-18,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:161 / 64 / 과체중(2...,"['GambiTab-D4_1통*3#405', 'GambiTab-D5_1통*3#405...",...,2.35,0.38,0.333,107.2,0.91,-6.5,-9.0,2.5,1225.0,68.0
128901,incheon,2785.0,"인천광역시 연수구 한나루로105번길 68 (옥련동, 현대4차아파트)",2024-01-11,16.0,2,NaN,2024-01-18,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:166 / 80 / 경도비만(...,"['GambiTab-A_1통*2#270', 'GambiTab-A_1통*2#270']",...,2.79,0.378,0.331,164.5,0.97,-18.7,-18.7,0.0,1367.0,62.0
128976,incheon,2821.0,"충청남도 천안시 동남구 청당4로 60 (청당동, 청당동 한양수자인 블루시티)",2024-01-25,56.0,2,NaN,2024-01-25,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:150 / 64 / 경도비만(...,NaN,...,1.95,0.38,0.333,139.2,0.92,-14.1,-14.7,0.6,1116.0,65.0


In [15]:
info_df = filtered_df.iloc[:,:8]
df = filtered_df.iloc[:,8:]

In [16]:
cols = df.columns.tolist()

In [17]:
len(cols)

810

In [18]:
# 컬럼 이름 분리하여 리스트 생성
split_columns = [col.rsplit('_', 1) for col in cols]
# 숫자와 문자열을 각각 리스트로 분리
index_level_1 = [int(x[1]) for x in split_columns]
index_level_2= ['Date', 'ProgressNote', 'MedicineName','Memo','체중','골격근량','체지방량','BMI','혈압(고)','혈압(저)','맥박수','체지방률','Height','근육량',
            'InterCellWater','ExtraCellWater','TotalBodyWater','ProteinMass','MineralMass','FatFreeMass','Osseus','ECW_TBW','ECF_TBF','VFA','WHR',
            'WeightControl','FatControl','MuscleControl','BMR','FitnessScore']*27

In [19]:
multi_index = pd.MultiIndex.from_arrays([index_level_1, index_level_2], names=['Number', 'Description'])

In [20]:
df.columns = multi_index

In [21]:
info_df.columns = pd.MultiIndex.from_arrays([[0 for i in range(len(info_df.columns))], info_df.columns], names=['Number', 'Description'])

In [22]:
info_df

Number             0                           \
Description   Region PatientID PatientChartNo   
2720         bundang     39817         118562   
8361         bundang    186479         122116   
14678        bundang    346681         122073   
15580        bundang    364561         122237   
15833        bundang    369511         124008   
...              ...       ...            ...   
128890       incheon     14470         2739.0   
128900       incheon     14535         2784.0   
128901       incheon     14540         2785.0   
128976       incheon     15021         2821.0   
129096       incheon     15776         2947.0   

Number                                                                    \
Description                               PatientAddr11 PatientFirstDate   
2720           경기도 성남시 분당구 판교로 393(삼평동, 봇들마을2단지이지더원아파트)       2023-03-25   
8361             경기도 수원시 영통구 동탄원천로881번길 35 (매탄동, 주공그린빌)       2023-09-02   
14678                     경기도 성남시 중원구 여수울로29번길 21 (여수동)       2023-08-30   
15580                   경기도 성남시 중원구 여수울로 51 (여수동, 산들마을)       2023-09-13   
15833             경기도 의왕시 바라산로 75 (학의동, 의왕백운 골드클래스 아파트)       2024-01-08   
...                                                 ...              ...   
128890             인천광역시 미추홀구 인주대로496번길 42 (주안동, 플러스캐슬)       2024-01-11   
128900            인천광역시 연수구 한나루로105번길 68 (옥련동, 현대4차아파트)       2024-01-11   
128901            인천광역시 연수구 한나루로105번길 68 (옥련동, 현대4차아파트)       2024-01-11   
128976       충청남도 천안시 동남구 청당4로 60 (청당동, 청당동 한양수자인 블루시티)       2024-01-25   
129096               인천광역시 남동구 호구포로899번길 8 (간석동, 두진아파트)       2024-02-13   

Number                                                 
Description   Age PatientSex              Description  
2720         60.0          2                      NaN  
8361         31.0          2  내원경로 이주희(강45909/ 핸7385)  
14678        43.0          2                      NaN  
15580        44.0          2                      NaN  
15833        36.0          1                      NaN  
...           ...        ...                      ...  
128890       49.0          2                      NaN  
128900       44.0          2                      NaN  
128901       16.0          2                      NaN  
128976       56.0          2                      NaN  
129096       59.0          1                      NaN  

[1573 rows x 8 columns]

In [23]:
filtered_recol = pd.concat([info_df,df],axis=1)

In [24]:
BF.save_excel(filtered_recol,'22.11.07~24.05.16_대면초진_3개월경과_재측정_표본')

In [31]:
FR_df=BF.FR_df
info_df = FR_df.iloc[:,:7]
df = FR_df.iloc[:,7:]

In [32]:
index_level_1 = ['First visit']*30+['After visit']*30
index_level_2= ['Date', 'ProgressNote', 'MedicineName','Memo','체중','골격근량','체지방량','BMI','혈압(고)','혈압(저)','맥박수','체지방률','Height','근육량',
            'InterCellWater','ExtraCellWater','TotalBodyWater','ProteinMass','MineralMass','FatFreeMass','Osseus','ECW_TBW','ECF_TBF','VFA','WHR',
            'WeightControl','FatControl','MuscleControl','BMR','FitnessScore']*2

In [33]:
index_level_1 = ['First visit']*30+['After visit']*30
index_level_2= ['Date', 'ProgressNote', 'MedicineName','Memo','체중','골격근량','체지방량','BMI','혈압(고)','혈압(저)','맥박수','체지방률','Height','근육량',
            'InterCellWater','ExtraCellWater','TotalBodyWater','ProteinMass','MineralMass','FatFreeMass','Osseus','ECW_TBW','ECF_TBF','VFA','WHR',
            'WeightControl','FatControl','MuscleControl','BMR','FitnessScore']*2
multi_index = pd.MultiIndex.from_arrays([index_level_1, index_level_2], names=['Number', 'Description'])

In [34]:
df.columns = multi_index

In [35]:
info_df.columns = pd.MultiIndex.from_arrays([[0 for i in range(7)], info_df.columns.tolist()], names=['Number', 'Description'])

In [36]:
FR_df = pd.concat([info_df,df],axis=1)

In [37]:
BF.save_excel(FR_df,'22.11.07~24.05.16_대면초진_3개월경과_재측정_전후비교')